In [1]:
import sys
from datetime import datetime
from os.path import join
from warnings import warn


import numpy as np
import pandas as pd
import scipy as sp

from utils import *

In [2]:
# sys.argv[1] = 'test'

In [3]:
dir_arg = sys.argv[1]
if dir_arg == '-f':
    file_dir = join('..', 'dataset', 'train')
else:
    file_dir = join('..', 'dataset',  dir_arg)

In [4]:
train_df = pd.read_pickle(join(file_dir, 'base_feauture.pkl'))

sample = pd.read_pickle(join(file_dir, 'hotelid.pkl'))

now_date = train_df.orderdate.max().date()
print(datetime.now(), now_date)

uid_shape, hotelid_shape, basicroomid_shape, roomid_shape = print_shape(
    train_df, ['uid', 'hotelid', 'basicroomid', 'roomid'])

2017-08-06 22:13:51.452623 2017-06-11
2017-08-06 22:13:51.452853
--------------------
uid uniuqe shape 29960
hotelid uniuqe shape 18543
basicroomid uniuqe shape 108611
roomid uniuqe shape 804370
--------------------


In [5]:
feature_path = join(file_dir, 'hotel_feature.pkl')
print(datetime.now(), 'begin', feature_path)

2017-08-06 22:13:51.534344 begin ../dataset/train/hotel_feature.pkl


## 添加基本特征

In [6]:
# sample = add_column(train_df, sample, 'hotelid', 'star')

In [7]:
# idf = pd.read_csv('models/06-1217-importance.txt', index_col=[0])

# nidf = idf.loc[idf.index.str.extract('^(hotelid)').notnull()]

# nidf

# nidf.loc[nidf['0']==0]

## 上下级关联统计特征

In [24]:
for f in ['basicroomid', 'roomid']:
    print(datetime.now(), 'begin hotel', f, 'count')
    sample = extract_feature_count('hotelid', f, train_df, sample)

2017-08-06 22:35:31.921879 begin hotel basicroomid count
2017-08-06 22:35:32.406088 begin hotel roomid count


In [25]:
for i in range(8):
    f = 'roomservice_%d' % (i+1)
    sample = extract_feature_count('hotelid', f, train_df, sample)

In [9]:
# get_corr(train_df, sample, 'hotelid')

In [10]:
for i in range(1, 4):
    f = 'roomtag_%d' % (i+1)
    sample = extract_feature_count('hotelid', f, train_df, sample)

In [11]:
# get_corr(train_df, sample, 'hotelid')

### 删除无历史记录却有历史返现值的记录值(默认值为200）

In [12]:
lastord_cols = [x for x in train_df.columns if x.endswith('lastord')]

In [13]:
train_df.loc[train_df.orderdate_lastord.isnull(), 'return_lastord'] = np.nan

In [14]:
train_df.price_deduct.describe()

count    1.094826e+06
mean     2.901501e+03
std      7.963338e+03
min      2.330000e+02
25%      1.010000e+03
50%      1.634000e+03
75%      2.987000e+03
max      7.802000e+05
Name: price_deduct, dtype: float64

## 基本特征 

## 显示的最终价格和原价格的特征 

In [17]:
use_describe = ['max', 'min', 'median', 'mean', 'std', 'nunique']

In [18]:
train_df['price_real'] = train_df['price_deduct'] + train_df['returnvalue']

In [19]:
sample = extract_value_describe_feature('hotelid', 'price_deduct', train_df, sample, use_describe)

sample = extract_value_describe_feature('hotelid', 'price_real', train_df, sample, ['max', 'mean', 'std'])

sample = extract_value_describe_feature('hotelid', 'returnvalue', train_df, sample, ['max', 'mean', 'median'])

In [20]:
# get_corr(train_df, sample, 'hotelid')

,orderlabel,hotelid_star,hotelid__basicroomid_count,hotelid__roomid_count,hotelid__roomservice_1_count,hotelid__roomservice_2_count,hotelid__roomservice_3_count,hotelid__roomservice_4_count,hotelid__roomservice_5_count,hotelid__roomservice_6_count,...,hotelid__price_deduct_median,hotelid__price_deduct_mean,hotelid__price_deduct_std,hotelid__price_deduct_nunique,hotelid__price_real_max,hotelid__price_real_mean,hotelid__price_real_std,hotelid__returnvalue_max,hotelid__returnvalue_mean,hotelid__returnvalue_median
orderlabel,1.000000,-0.062683,-0.068750,-0.043375,0.006582,-0.037106,-0.068868,-0.043301,-0.049484,-0.029382,...,-0.044040,-0.037606,-0.021799,-0.047352,-0.023799,-0.037724,-0.021834,-0.024261,-0.032980,0.010949
hotelid_star,-0.062683,1.000000,0.252953,0.296967,-0.242361,0.341261,0.623081,0.232470,0.430181,0.172103,...,0.529667,0.445887,0.238126,0.356056,0.244961,0.446588,0.238603,0.239270,0.346798,-0.034211
hotelid__basicroomid_count,-0.068750,0.252953,1.000000,0.214686,0.059857,0.176144,0.257764,0.280136,0.158051,0.066047,...,0.228866,0.229595,0.193974,0.258516,0.231643,0.229409,0.193156,0.126894,0.143503,-0.016067
hotelid__roomid_count,-0.043375,0.296967,0.214686,1.000000,-0.024507,0.248710,0.393166,0.281889,0.159834,0.018552,...,0.462193,0.323995,0.133606,0.937836,0.160712,0.323050,0.133345,0.189142,0.158839,-0.056305
hotelid__roomservice_1_count,0.006582,-0.242361,0.059857,-0.024507,1.000000,-0.107726,-0.169526,0.030812,-0.091688,0.019625,...,-0.143184,-0.121319,-0.063570,-0.029932,-0.065751,-0.121223,-0.063283,-0.031351,-0.075950,-0.006667
hotelid__roomservice_2_count,-0.037106,0.341261,0.176144,0.248710,-0.107726,1.000000,0.354003,0.213727,0.153776,-0.052840,...,0.392524,0.329855,0.179388,0.302463,0.187482,0.329539,0.179446,0.180019,0.203134,-0.058633
hotelid__roomservice_3_count,-0.068868,0.623081,0.257764,0.393166,-0.169526,0.354003,1.000000,0.366548,0.345327,0.153429,...,0.524186,0.443350,0.247537,0.469853,0.261507,0.443090,0.247758,0.257329,0.283479,-0.091631
hotelid__roomservice_4_count,-0.043301,0.232470,0.280136,0.281889,0.030812,0.213727,0.366548,1.000000,0.159576,0.043276,...,0.249696,0.235645,0.154809,0.318985,0.157527,0.235687,0.155118,0.162950,0.162266,-0.035404
hotelid__roomservice_5_count,-0.049484,0.430181,0.158051,0.159834,-0.091688,0.153776,0.345327,0.159576,1.000000,0.216441,...,0.236777,0.195124,0.101963,0.192187,0.107702,0.195480,0.102182,0.109162,0.154921,-0.027853
hotelid__roomservice_6_count,-0.029382,0.172103,0.066047,0.018552,0.019625,-0.052840,0.153429,0.043276,0.216441,1.000000,...,-0.031340,-0.022173,-0.008793,-0.001457,-0.006453,-0.022258,-0.008466,0.015435,-0.020510,-0.054588


### 房间的面积统计特征

###  删掉为负的值

In [21]:
train_df.loc[train_df.basic_minarea<0, 'basic_minarea'] = np.nan

In [22]:
# sample = extract_value_describe_feature('hotelid', 'basic_minarea', train_df, sample, ['max', 'mean', 'median'])

# sample = extract_value_describe_feature('hotelid', 'basic_maxarea', train_df, sample, ['min', 'mean', 'median'])

In [23]:
# get_corr(train_df, sample, 'hotelid').tail(25)

,orderlabel,hotelid_star,hotelid__basicroomid_count,hotelid__roomid_count,hotelid__roomservice_1_count,hotelid__roomservice_2_count,hotelid__roomservice_3_count,hotelid__roomservice_4_count,hotelid__roomservice_5_count,hotelid__roomservice_6_count,...,hotelid__price_real_std,hotelid__returnvalue_max,hotelid__returnvalue_mean,hotelid__returnvalue_median,hotelid__basic_minarea_max,hotelid__basic_minarea_mean,hotelid__basic_minarea_median,hotelid__basic_maxarea_min,hotelid__basic_maxarea_mean,hotelid__basic_maxarea_median
hotelid__roomservice_5_count,-0.049484,0.430181,0.158051,0.159834,-0.091688,0.153776,0.345327,0.159576,1.000000,0.216441,...,0.102182,0.109162,0.154921,-0.027853,0.085319,0.167648,0.168524,0.213480,0.175920,0.179758
hotelid__roomservice_6_count,-0.029382,0.172103,0.066047,0.018552,0.019625,-0.052840,0.153429,0.043276,0.216441,1.000000,...,-0.008466,0.015435,-0.020510,-0.054588,-0.019642,-0.031131,-0.018017,0.000630,-0.024233,-0.007283
hotelid__roomservice_7_count,-0.055603,0.159056,0.074753,0.063591,-0.028534,0.181117,0.131791,0.087337,0.157338,0.277922,...,0.034561,0.033986,0.044528,-0.099572,0.037933,0.066961,0.064213,0.074583,0.070731,0.069653
hotelid__roomservice_8_count,-0.088007,0.345644,0.160045,0.159380,-0.088065,0.188064,0.311626,0.155787,0.194913,0.080069,...,0.103891,0.139929,0.282462,-0.031200,0.098121,0.184923,0.182385,0.192773,0.185939,0.183973
hotelid__roomtag_2_count,-0.047114,0.399008,0.194770,0.617042,-0.065906,0.352976,0.505520,0.278013,0.231101,0.075065,...,0.223040,0.264638,0.263284,-0.088470,0.155778,0.232373,0.193842,0.261752,0.238301,0.200413
hotelid__roomtag_3_count,-0.053435,0.438370,0.165632,0.573758,-0.074051,0.378808,0.520856,0.250313,0.253435,0.072827,...,0.227350,0.269315,0.287535,-0.105151,0.099525,0.198465,0.186851,0.273895,0.203586,0.192141
hotelid__roomtag_4_count,-0.072672,0.371318,0.158013,0.148807,-0.093739,0.164700,0.291765,0.159676,0.198470,0.067737,...,0.097335,0.123649,0.271078,-0.007117,0.106327,0.184985,0.173913,0.190299,0.188529,0.178850
hotelid__price_deduct_max,-0.023642,0.243218,0.233155,0.159056,-0.066571,0.186362,0.259215,0.155252,0.106874,-0.007117,...,0.954847,0.545404,0.452837,-0.024663,0.622825,0.499151,0.282535,0.300893,0.496466,0.282074
hotelid__price_deduct_min,-0.038286,0.538091,0.158300,0.343817,-0.168377,0.369081,0.493371,0.197955,0.231936,-0.016365,...,0.470887,0.413478,0.498780,-0.081104,0.224045,0.418580,0.414978,0.508458,0.424257,0.424608
hotelid__price_deduct_median,-0.044040,0.529667,0.228866,0.462193,-0.143184,0.392524,0.524186,0.249696,0.236777,-0.031340,...,0.515101,0.443642,0.527073,-0.079595,0.261131,0.450676,0.426873,0.481900,0.457009,0.435987


## 过去物理房型和子房型的统计特征 

In [32]:
# get_corr(train_df, sample, 'hotelid')

In [33]:
#     'basic_week_ordernum_ratio', 'basic_30days_ordnumratio',   'basic_comment_ratio',  not use
basic_cols = [
    'basic_30days_realratio'
]

In [34]:
stat_describe = ['min', 'mean', 'max']

In [35]:
for col in basic_cols:
    sample = extract_value_describe_feature('hotelid', col, train_df, sample, stat_describe)

In [36]:
sample = extract_value_describe_feature('hotelid', 'basic_week_ordernum_ratio', train_df, sample, ['count'])

In [ ]:
sample = extract_value_describe_feature('hotelid', 'basic_recent3_ordernum_ratio', train_df, sample, ['max'])

In [37]:
room_cols = ['room_30days_realratio'] # 'room_30days_ordnumratio', 

In [38]:
for col in room_cols:
    sample = extract_value_describe_feature('hotelid', col, train_df, sample, ['max', 'min', 'mean'])

In [39]:
sample = extract_value_describe_feature('hotelid', 'room_30days_ordnumratio', train_df, sample, ['count'])

In [40]:
name_fmt = '{}_diff_{}'.format('hotelid', '{}')

price_diff_name = name_fmt.format('price_last_lastord')
hotel_minprice_diff_name = name_fmt.format('hotel_minprice_lastord')
basic_minprice_diff_name = name_fmt.format('basic_minprice_lastord')

In [41]:
train_df[price_diff_name] = train_df['price_deduct'] - train_df['price_last_lastord']
train_df[hotel_minprice_diff_name] = train_df['price_deduct'] - train_df['hotel_minprice_lastord']
train_df[basic_minprice_diff_name] = train_df['price_deduct'] - train_df['basic_minprice_lastord']

In [42]:
price_desr = ['mean', 'max', 'min']

In [43]:
sample = extract_value_describe_feature('hotelid', price_diff_name, train_df, sample, price_desr)
sample = extract_value_describe_feature('hotelid', hotel_minprice_diff_name, train_df, sample, price_desr)
sample = extract_value_describe_feature('hotelid', basic_minprice_diff_name, train_df, sample, price_desr)

In [44]:
# get_corr(train_df, sample, 'hotelid').tail(24)

## 历史价格与现在差价统计特征

In [45]:
hotel_lastord = train_df[[
    'uid', 'hotelid_lastord', 'hotelid', 'star_lastord',
    'hotel_minprice_lastord', 'orderdate_lastord'
]].drop_duplicates()

In [46]:
if uid_shape != hotel_lastord.shape[0]:
    warn('uid_shape not equal [uid ,hotelid_lastord, hotelid]')

/home/zhanglun/.pyenv/versions/anaconda3-4.1.0/lib/python3.5/site-packages/ipykernel/__main__.py:2: UserWarning: uid_shape not equal [uid ,hotelid_lastord, hotelid]
  from ipykernel import kernelapp as app


## 历史购买时间间隔统计特征 

In [47]:
span_name, t = '{}_span'.format('hotelid'), 'hotelid'

In [48]:
# train_df[span_name] = (now_date - train_df.orderdate_lastord).dt.days

# sample = extract_value_describe_feature(t, span_name, train_df, sample, ['max', 'min', 'mean'])

In [49]:
# get_corr(train_df, sample, 'hotelid').tail(8)

## 用户过去是否购买或者没有记录

In [50]:
sample = extract_lastord_is_nan(hotel_lastord, sample, 'hotelid', 'hotelid_lastord')

In [51]:
# sample = extract_is_lastord(hotel_lastord, sample, 'hotelid', 'hotelid_lastord')

In [52]:
# get_corr(train_df, sample, 'hotelid').tail()

In [53]:
def extract_lastord_feature_max_min(t, hotel_lastord, sample):
    min_fmt = '{}_min'.format(t)
    max_fmt = '{}_max'.format(t)
    series = hotel_lastord.groupby('hotelid')[t].min()
    series.name = min_fmt
    sample = sample.join(series, on='hotelid')
    series = hotel_lastord.groupby('hotelid')[t].max()
    series.name = max_fmt
    sample = sample.join(series, on='hotelid')
    max_equal_min_fmt = '{}_max_equeal_min'.format(t)
    sample[max_equal_min_fmt] = (sample[max_fmt] == sample[min_fmt]).astype(np.int8)
    sample = press_date(sample, [max_equal_min_fmt, max_fmt, min_fmt])
    return sample

In [54]:
# sample = extract_lastord_feature_max_min('star_lastord', hotel_lastord, sample)

# sample = extract_lastord_feature_max_min('hotel_minprice_lastord', hotel_lastord, sample)

# hotel_lastord['orderdate_lastord_days'] = (now_date - hotel_lastord.orderdate_lastord).dt.days

# sample = extract_lastord_feature_max_min('orderdate_lastord_days', hotel_lastord, sample)

In [55]:
# get_corr(train_df, sample, 'hotelid')

In [56]:
# sample.info()

In [57]:
sample.to_pickle(feature_path)

print(datetime.now(), 'save to', feature_path)

2017-08-05 07:24:47.089893 save to ../dataset/train/hotel_feature.pkl
